## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os, sys
import numpy as np
# Import API key
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("config.py"))))
import config
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Abu Samrah,35.3029,37.1841,63.19,29,0,6.53,SY,2022-05-21 01:05:46,clear sky
1,1,Anadyr,64.7500,177.4833,35.01,80,63,5.99,RU,2022-05-21 01:05:46,broken clouds
2,2,Bayan,46.0833,127.4000,70.21,48,22,11.59,CN,2022-05-21 01:05:47,few clouds
3,3,Albany,42.6001,-73.9662,69.57,79,100,1.01,US,2022-05-21 01:04:08,overcast clouds
4,4,Bredasdorp,-34.5322,20.0403,50.07,83,14,6.08,ZA,2022-05-21 01:05:47,few clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))


What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
5,5,Tomigusuku,26.1850,127.6750,77.14,88,75,12.66,JP,2022-05-21 01:05:48,broken clouds
7,7,San Patricio,28.0170,-97.5169,84.29,74,0,22.08,US,2022-05-21 01:05:49,clear sky
13,13,Hami,42.8000,93.4500,77.56,25,2,9.57,CN,2022-05-21 01:05:51,clear sky
17,17,Buchanan,5.8808,-10.0467,76.73,90,100,6.29,LR,2022-05-21 01:05:53,moderate rain
19,19,Hilo,19.7297,-155.0900,84.27,71,75,9.22,US,2022-05-21 01:05:03,broken clouds
21,21,Kapaa,22.0752,-159.3190,80.58,88,75,14.97,US,2022-05-21 01:05:54,broken clouds
27,27,Ixtapa,20.7000,-105.2000,78.22,69,0,9.22,MX,2022-05-21 01:05:57,clear sky
28,28,Canoa,18.3550,-71.1585,76.82,85,65,4.25,DO,2022-05-21 01:05:57,broken clouds
35,35,Victoria,22.2855,114.1577,80.94,82,22,1.99,HK,2022-05-21 01:06:00,few clouds
36,36,Cartagena,10.3997,-75.5144,83.82,79,75,3.44,CO,2022-05-21 01:06:00,broken clouds


In [38]:
# 4a. Determine if there are any empty rows.
print(np.where(pd.isnull(preferred_cities_df)))

(array([], dtype=int64), array([], dtype=int64))


In [58]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.drop(preferred_cities_df.loc[preferred_cities_df["City"]==""].index, inplace=True)

In [59]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
5,Tomigusuku,JP,77.14,broken clouds,26.1850,127.6750,
7,San Patricio,US,84.29,clear sky,28.0170,-97.5169,
13,Hami,CN,77.56,clear sky,42.8000,93.4500,
17,Buchanan,LR,76.73,moderate rain,5.8808,-10.0467,
19,Hilo,US,84.27,broken clouds,19.7297,-155.0900,
21,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
27,Ixtapa,MX,78.22,clear sky,20.7000,-105.2000,
28,Canoa,DO,76.82,broken clouds,18.3550,-71.1585,
35,Victoria,HK,80.94,few clouds,22.2855,114.1577,
36,Cartagena,CO,83.82,broken clouds,10.3997,-75.5144,


In [60]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [61]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df.loc[hotel_df["Hotel Name"]==""].index, inplace=True)
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
5,Tomigusuku,JP,77.14,broken clouds,26.1850,127.6750,Hotel Gran View Okinawa
7,San Patricio,US,84.29,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
13,Hami,CN,77.56,clear sky,42.8000,93.4500,Laoyutang Resort
17,Buchanan,LR,76.73,moderate rain,5.8808,-10.0467,Kamaneahn Hotel Inc
19,Hilo,US,84.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
707,Quibdo,CO,79.14,broken clouds,5.6947,-76.6611,Domus Goval Hotel Quibdó
712,Tevaitoa,PF,79.75,clear sky,-16.7833,-151.5000,Hôtel Raiatea Lodge
715,Janakpur,NP,81.03,clear sky,26.7183,85.9065,Hotel Sita Palace
716,Denpasar,ID,85.66,few clouds,-8.6500,115.2167,OYO 3156 Hotel Warta 2


In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))